<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Colour_Describer_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Delta-E Calculator and Describer for Accurate Description of Getty Art and Architecture Thesaurus Faceted Headings for Colours

This notebook calculates the Euclidean distance between two colours and displays the nearest description for that colour as controlled vocabulary from the [Getty Art and Architecture Thesaurus](https://www.getty.edu/research/tools/vocabularies/aat/)

**Instructions for use**



*   [Download](https://github.com/hsandaver/hsandaver/blob/main/iscc_nbs_lab_colors.csv) the required CSV. You can also use your own pallette with your own vocabulary, however it may break the code. Use at your own risk.
*   The file you submit must be titled iscc_nbs_lab_colors.csv even if the content of the file is not ISCC NBS



In [ ]:
# Install required libraries
!pip install colormath plotly

# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from math import sqrt
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from google.colab import files
import io

# Function to upload and load the dataset
def upload_dataset():
    """
    Prompts the user to upload the ISCC-NBS LAB colors CSV file.

    Returns:
        pd.DataFrame: DataFrame containing the ISCC-NBS LAB colors.
    """
    uploaded = files.upload()
    if 'iscc_nbs_lab_colors.csv' not in uploaded:
        raise FileNotFoundError("Please upload 'iscc_nbs_lab_colors.csv'.")

    # Read the uploaded CSV file into a DataFrame
    df = pd.read_csv(io.BytesIO(uploaded['iscc_nbs_lab_colors.csv']))
    return df

# Function to validate the dataset
def validate_dataset(df):
    """
    Validates that the DataFrame contains the necessary columns.

    Parameters:
        df (pd.DataFrame): The DataFrame to validate.

    Raises:
        ValueError: If required columns are missing.
    """
    required_columns = {'L', 'A', 'B', 'Color Name'}
    if not required_columns.issubset(df.columns):
        missing = required_columns - set(df.columns)
        raise ValueError(f"CSV file is missing required columns: {missing}")

# Function to calculate Delta-E (CIE76) using vectorized operations
def calculate_delta_e(input_lab, dataset_df):
    """
    Calculates the Delta-E (CIE76) between an input LAB color and all colors in the dataset.

    Parameters:
        input_lab (list or np.ndarray): The input LAB color as [L, A, B].
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.

    Returns:
        pd.Series: Series containing Delta-E values for each color in the dataset.
    """
    delta_e = np.sqrt(
        (dataset_df['L'] - input_lab[0]) ** 2 +
        (dataset_df['A'] - input_lab[1]) ** 2 +
        (dataset_df['B'] - input_lab[2]) ** 2
    )
    return delta_e

# Function to find the closest color
def find_closest_color(input_lab, dataset_df):
    """
    Finds the closest color in the dataset to the input LAB color based on Delta-E.

    Parameters:
        input_lab (list or np.ndarray): The input LAB color as [L, A, B].
        dataset_df (pd.DataFrame): DataFrame containing the dataset LAB colors.

    Returns:
        tuple: (Closest color row as pd.Series, Minimum Delta-E value)
    """
    delta_e_values = calculate_delta_e(input_lab, dataset_df)
    min_index = delta_e_values.idxmin()
    min_delta_e = delta_e_values[min_index]
    closest_color = dataset_df.loc[min_index]
    return closest_color, min_delta_e

# Function to convert LAB to RGB
def lab_to_rgb(lab_color):
    """
    Converts a LAB color to RGB.

    Parameters:
        lab_color (list or tuple): The LAB color as [L, A, B].

    Returns:
        tuple: RGB color as (R, G, B) with values between 0 and 255.
    """
    lab = LabColor(lab_l=lab_color[0], lab_a=lab_color[1], lab_b=lab_color[2])
    rgb = convert_color(lab, sRGBColor, target_illuminant='d65')

    # Clamp the RGB values to [0, 1]
    rgb_clamped = [
        min(max(rgb.rgb_r, 0), 1),
        min(max(rgb.rgb_g, 0), 1),
        min(max(rgb.rgb_b, 0), 1)
    ]

    # Convert to 0-255 range and return as integers
    return tuple(int(c * 255) for c in rgb_clamped)

# Function to validate input LAB color
def validate_lab_color(lab):
    """
    Validates the input LAB color.

    Parameters:
        lab (list or tuple): The LAB color as [L, A, B].

    Raises:
        ValueError: If the LAB color is invalid.
    """
    if not isinstance(lab, (list, tuple, np.ndarray)) or len(lab) != 3:
        raise ValueError("Input LAB color must be a list, tuple, or array of three numerical values.")

    L, A, B = lab
    if not (0 <= L <= 100):
        raise ValueError("L component must be between 0 and 100.")
    if not (-128 <= A <= 127):
        raise ValueError("A component must be between -128 and 127.")
    if not (-128 <= B <= 127):
        raise ValueError("B component must be between -128 and 127.")

# Enhanced Function to create the color comparison plot
def create_color_comparison_plot(input_rgb, closest_rgb, input_lab, closest_lab, closest_color_name, delta_e):
    """
    Creates an interactive Plotly scatter plot displaying input and closest colors side by side.

    Parameters:
        input_rgb (tuple): RGB values of the input color.
        closest_rgb (tuple): RGB values of the closest match.
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.
        delta_e (float): Delta-E value between input and closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    # Prepare data for the scatter plot
    colors = ['Input Color', 'Closest Match']
    rgb_values = [f'rgb({input_rgb[0]}, {input_rgb[1]}, {input_rgb[2]})',
                 f'rgb({closest_rgb[0]}, {closest_rgb[1]}, {closest_rgb[2]})']
    hover_texts = [
        f'Input Color<br>LAB: {input_lab}',
        f'Closest Match: {closest_color_name}<br>LAB: {closest_lab}<br>Delta-E: {delta_e:.2f}'
    ]

    # Create the figure
    color_fig = go.Figure()

    # Add scatter points as color swatches
    for i, (color, rgb, hover) in enumerate(zip(colors, rgb_values, hover_texts)):
        color_fig.add_trace(go.Scatter(
            x=[i],
            y=[1],
            mode='markers',
            marker=dict(
                size=200,
                color=rgb,
                line=dict(width=2, color='black')
            ),
            name=color,
            hoverinfo='text',
            hovertext=hover
        ))

    # Update layout for aesthetics
    color_fig.update_layout(
        title='Input Color vs Closest ISCC-NBS Color',
        xaxis=dict(
            showticklabels=False,
            showgrid=False,
            zeroline=False
        ),
        yaxis=dict(
            showticklabels=False,
            showgrid=False,
            zeroline=False
        ),
        showlegend=False,
        template='plotly_dark',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    return color_fig

# Enhanced Function to create the LAB comparison plot
def create_lab_comparison_plot(input_lab, closest_lab, closest_color_name):
    """
    Creates an interactive Plotly grouped bar plot comparing LAB values of input and closest colors.

    Parameters:
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    components = ['L', 'A', 'B']

    lab_fig = go.Figure()

    # Add Input LAB values
    lab_fig.add_trace(go.Bar(
        x=components,
        y=input_lab,
        name='Input LAB',
        marker_color='rgba(0, 123, 255, 0.7)',  # Blue with transparency
        hoverinfo='text',
        hovertext=[f'L: {input_lab[0]}', f'A: {input_lab[1]}', f'B: {input_lab[2]}']
    ))

    # Add Closest LAB values
    lab_fig.add_trace(go.Bar(
        x=components,
        y=closest_lab,
        name=f'Closest LAB: {closest_color_name}',
        marker_color='rgba(40, 167, 69, 0.7)',  # Green with transparency
        hoverinfo='text',
        hovertext=[f'L: {closest_lab[0]}', f'A: {closest_lab[1]}', f'B: {closest_lab[2]}']
    ))

    # Update layout for better aesthetics
    lab_fig.update_layout(
        title='LAB Value Comparison',
        xaxis_title='LAB Components',
        yaxis_title='Values',
        barmode='group',
        template='plotly_dark',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        margin=dict(l=50, r=50, t=80, b=50)
    )

    # Enhance y-axis with gridlines and better tick formatting
    lab_fig.update_yaxes(showgrid=True, gridcolor='gray', zeroline=False)

    return lab_fig

# Function to create a 3D LAB color space plot
def create_lab_color_space_plot(input_lab, closest_lab, closest_color_name):
    """
    Creates an interactive 3D Plotly scatter plot visualizing the input and closest colors in LAB space.

    Parameters:
        input_lab (list): LAB values of the input color.
        closest_lab (list): LAB values of the closest match.
        closest_color_name (str): Name of the closest color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    lab_fig = go.Figure()

    # Add Input Color
    lab_fig.add_trace(go.Scatter3d(
        x=[input_lab[0]],
        y=[input_lab[1]],
        z=[input_lab[2]],
        mode='markers+text',
        marker=dict(
            size=8,
            color='blue',
            opacity=0.8
        ),
        text=['Input Color'],
        textposition='top center',
        name='Input Color'
    ))

    # Add Closest Color
    lab_fig.add_trace(go.Scatter3d(
        x=[closest_lab[0]],
        y=[closest_lab[1]],
        z=[closest_lab[2]],
        mode='markers+text',
        marker=dict(
            size=8,
            color='green',
            opacity=0.8
        ),
        text=[f'Closest: {closest_color_name}'],
        textposition='top center',
        name='Closest Color'
    ))

    # Optionally, add lines or additional colors from the dataset for context
    # For simplicity, only plotting input and closest colors

    # Update layout for better visualization
    lab_fig.update_layout(
        title='3D LAB Color Space Visualization',
        scene=dict(
            xaxis_title='L',
            yaxis_title='A',
            zaxis_title='B',
            xaxis=dict(range=[0, 100]),
            yaxis=dict(range=[-128, 127]),
            zaxis=dict(range=[-128, 127]),
            bgcolor='rgb(20, 20, 20)',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        template='plotly_dark',
        margin=dict(l=0, r=0, t=80, b=0)
    )

    return lab_fig

# Function to display results in a table
def display_results_table(input_lab, closest_color_name, delta_e, closest_lab):
    """
    Displays the results in a tabular format.

    Parameters:
        input_lab (list): LAB values of the input color.
        closest_color_name (str): Name of the closest color.
        delta_e (float): Delta-E value.
        closest_lab (list): LAB values of the closest color.
    """
    data = {
        'Description': ['Input LAB Color', 'Closest ISCC-NBS Color', 'Delta-E Value', 'LAB of Closest Color'],
        'Value': [input_lab, closest_color_name, f"{delta_e:.2f}", closest_lab]
    }
    results_df = pd.DataFrame(data)
    display(results_df)

# Function to display aggregated results in a table
def display_aggregated_results(results):
    """
    Displays the aggregated results for all input LAB colors in a table.

    Parameters:
        results (list of dict): List containing result dictionaries for each input color.
    """
    data = {
        'Input LAB': [result['Input LAB'] for result in results],
        'Closest ISCC-NBS Color': [result['Closest Color Name'] for result in results],
        'Delta-E': [f"{result['Delta-E']:.2f}" for result in results],
        'Closest LAB': [result['Closest LAB'] for result in results],
        'Input RGB': [result['Input RGB'] for result in results],
        'Closest RGB': [result['Closest RGB'] for result in results],
    }
    results_df = pd.DataFrame(data)
    display(results_df)

# Combined Color Comparison Plot
def create_combined_color_comparison_plot(results):
    """
    Creates an interactive Plotly scatter plot displaying all input and closest colors side by side.

    Parameters:
        results (list of dict): List containing result dictionaries for each input color.

    Returns:
        plotly.graph_objects.Figure: The Plotly figure object.
    """
    colors = []
    rgb_values = []
    hover_texts = []

    for idx, result in enumerate(results, start=1):
        colors.extend([f'Input Color {idx}', f'Closest Match {idx}'])
        rgb_values.extend([
            f'rgb({result["Input RGB"][0]}, {result["Input RGB"][1]}, {result["Input RGB"][2]})',
            f'rgb({result["Closest RGB"][0]}, {result["Closest RGB"][1]}, {result["Closest RGB"][2]})'
        ])
        hover_texts.extend([
            f'Input Color {idx}<br>LAB: {result["Input LAB"]}',
            f'Closest Match {idx}: {result["Closest Color Name"]}<br>LAB: {result["Closest LAB"]}<br>Delta-E: {result["Delta-E"]}'
        ])

    # Create the figure
    color_fig = go.Figure()

    # Add scatter points as color swatches
    for i, (color, rgb, hover) in enumerate(zip(colors, rgb_values, hover_texts)):
        color_fig.add_trace(go.Scatter(
            x=[i],
            y=[1],
            mode='markers',
            marker=dict(
                size=200,
                color=rgb,
                line=dict(width=2, color='black')
            ),
            name=color,
            hoverinfo='text',
            hovertext=hover
        ))

    # Update layout for aesthetics
    color_fig.update_layout(
        title='Input Colors vs Closest ISCC-NBS Colors',
        xaxis=dict(
            showticklabels=False,
            showgrid=False,
            zeroline=False
        ),
        yaxis=dict(
            showticklabels=False,
            showgrid=False,
            zeroline=False
        ),
        showlegend=False,
        template='plotly_dark',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    return color_fig

# Main Execution Flow

# Step 1: Upload and load the dataset
try:
    iscc_nbs_df = upload_dataset()
    print("📂 Dataset uploaded successfully.")
except Exception as e:
    print(f"❌ Error uploading dataset: {e}")

# Step 2: Validate the dataset
try:
    validate_dataset(iscc_nbs_df)
    print("✅ Dataset validation passed.")
except ValueError as ve:
    print(f"❌ Dataset validation error: {ve}")

# Step 3: Define and validate the input LAB colors
input_lab_colors = [
    [50, 0, 0],      # Example LAB input 1
    [60, 20, -10],   # Example LAB input 2
    [30, -20, 15],   # Example LAB input 3
    # Add more colors as needed
]

# Validate multiple input LAB colors
for idx, lab in enumerate(input_lab_colors, start=1):
    try:
        validate_lab_color(lab)
        print(f"🟢 Input LAB color {idx} is valid: {lab}")
    except ValueError as ve:
        print(f"❌ Input LAB validation error for color {idx}: {ve}")

# Step 4: Find the closest colors and process each input LAB color
results = []

for idx, input_lab_color in enumerate(input_lab_colors, start=1):
    print(f"\nProcessing Input LAB Color {idx}: {input_lab_color}")

    try:
        # Find the closest color
        closest_color, delta_e_value = find_closest_color(input_lab_color, iscc_nbs_df)

        # Extract details from the closest color
        closest_color_name = closest_color['Color Name']
        closest_lab_color = [closest_color['L'], closest_color['A'], closest_color['B']]

        print(f"🎨 Closest ISCC-NBS Color: {closest_color_name}")
        print(f"📏 Delta-E Value: {delta_e_value:.2f}")
        print(f"🔍 LAB of Closest Color: {closest_lab_color}")

        # Convert LAB to RGB
        input_rgb = lab_to_rgb(input_lab_color)
        closest_rgb = lab_to_rgb(closest_lab_color)

        print(f"🎨 Input RGB Color: {input_rgb}")
        print(f"🎨 Closest RGB Color: {closest_rgb}")

        # Create and display the color comparison plot
        color_comparison_fig = create_color_comparison_plot(
            input_rgb, closest_rgb, input_lab_color, closest_lab_color, closest_color_name, delta_e_value
        )
        color_comparison_fig.show()

        # Create and display the LAB comparison plot
        lab_comparison_fig = create_lab_comparison_plot(
            input_lab_color, closest_lab_color, closest_color_name
        )
        lab_comparison_fig.show()

        # Create and display the 3D LAB color space plot
        lab_color_space_fig = create_lab_color_space_plot(
            input_lab_color, closest_lab_color, closest_color_name
        )
        lab_color_space_fig.show()

        # Collect results
        results.append({
            'Input LAB': input_lab_color,
            'Closest Color Name': closest_color_name,
            'Delta-E': delta_e_value,
            'Closest LAB': closest_lab_color,
            'Input RGB': input_rgb,
            'Closest RGB': closest_rgb
        })

    except Exception as e:
        print(f"❌ Error processing input LAB color {idx}: {e}")

# Step 5: Display the aggregated results table
try:
    display_aggregated_results(results)
except Exception as e:
    print(f"❌ Error displaying aggregated results table: {e}")

# Step 6: (Optional) Create and display the combined color comparison plot
try:
    combined_color_comparison_fig = create_combined_color_comparison_plot(results)
    combined_color_comparison_fig.show()
except Exception as e:
    print(f"❌ Error creating combined color comparison plot: {e}")
